In [107]:
import pandas as pd

import json


In [114]:
df = pd.read_csv('../data/similarities_1.0.tsv',sep='\t')

In [115]:
df.sort_values(by='Similarity',ascending=False)

,ISO_1,LangName_1,ISO_2,LangName_2,Similarity,Robustness
338,ady,Adyghe,kbd,Kabardian,100.000,Low
7689,dak,Dakota,lkt,Lakota,89.250,Low
14095,ind,Indonesian,msa,Malay,70.725,High
22395,smo,Samoan,tvl,Tuvalu,70.639,Low
19464,msa,Malay,zsm,Standard Malay,43.057,High
...,...,...,...,...,...,...
22329,slv,Slovenian,zho,Chinese,0.005,High
19463,msa,Malay,zho,Chinese,0.005,High
10404,fin,Finnish,zho,Chinese,0.005,High
10601,fra,French,zho,Chinese,0.005,High


In [116]:
len(set(df[df['Similarity']>6.5]['ISO_1'])),len(df[df['Similarity']>6.5])

(124, 358)

In [117]:
len(set(df['ISO_1'].values))

312

In [152]:
language_list = len(set(df['ISO_2'].values).union(set(df['ISO_1'].values)))

In [118]:
df2 = pd.read_csv('../data/internet_languages.tsv',sep='\t').replace({'Dutch, Flemish':'Dutch'}).replace({'Catalan, Valencian':'Catalan'})
df2['Percentage'] = df2['Percentage'].str[:-1]

In [119]:
df2

,Rank,Language,Percentage
0,1,English,57.2
1,2,Russian,5.3
2,3,Spanish,4.6
3,4,French,4.1
4,5,German,3.9
5,6,Japanese,3.3
6,7,Turkish,2.5
7,8,Persian,2.1
8,9,Portuguese,1.9
9,10,Italian,1.7


In [120]:
langs = set(df2['Language'].values)

In [153]:
def create_node_dataset(df, df2):
    scores = []
    languages =  list(set(df['LangName_1'].values).union(set(df['LangName_2'].values)))
    for lang in languages:
        if lang in set(df2['Language'].values):
            scores.append(float(df2[df2['Language'] == lang]['Percentage'].values[0]))
        else:
            scores.append(0)
    
        
    return pd.DataFrame([languages,scores]).transpose().rename(columns={0:'language',1:'internet'})

create_node_dataset(df, df2)['internet'].sum()

98.10000000000001

In [154]:
node_df = create_node_dataset(df, df2)

In [162]:
node_df

,language,internet
0,Yakut,0
1,Fiji Hindi,0
2,Breton,0
3,Sranan Tongo,0
4,Ket,0
...,...,...
317,Shona,0
318,Eshtehardi,0
319,Tachelhit,0
320,Assamese,0


In [161]:
links_df = df[df['Similarity']>6.5]

In [157]:
links_df

,ISO_1,LangName_1,ISO_2,LangName_2,Similarity,Robustness
78,abk,Abkhazian,abq,Abaza,14.485,Low
271,abq,Abaza,ady,Adyghe,8.911,Low
274,abq,Abaza,ava,Avaric,9.055,Low
278,abq,Abaza,inh,Ingush,8.754,Low
289,ace,Achinese,aoz,Uab Meto,8.373,Low
...,...,...,...,...,...,...
22891,tat,Tatar,tuk,Turkmen,6.885,Medium
23083,ton,Tonga,tvl,Tuvalu,18.623,Low
23176,tvl,Tuvalu,war,Waray,6.877,Low
23279,vep,Veps,vro,Võro,10.161,Low


In [158]:
node_set = set(node_df['language'].values).intersection(set(links_df['LangName_2'].values))

In [159]:
node_set = set(node_df['language'].values).intersection(set(links_df['LangName_1'].values)).intersection(set(links_df['LangName_2'].values))


In [160]:
len(node_set)

81

In [163]:
# node_df = node_df[node_df['language'].isin(node_set)]

In [164]:
def create_json(df, df2):
    links = []
    nodes = []
    for idx in range(len(df)):
        links.append({'source': df.iloc[idx]['LangName_1'], 
            'target': df.iloc[idx]['LangName_2'],
            'sim': df.iloc[idx]['Similarity'] ,
        })

    for idx in range(len(df2)):
        nodes.append(
            {'id':df2.iloc[idx]['language'],
            'size':df2.iloc[idx]['internet']}
        )

    update_nodes = [dict(item, **{'type':'digital'}) if item['id'] in langs else dict(item, **{'type':'aural'}) for item in nodes]

    dumps = json.dumps( {'nodes':update_nodes,'links':links}, indent=4)  

    with open("../data/internet_languages.json", "w") as outfile:
        outfile.write(dumps)

    return {'nodes':update_nodes,'links':links}


create_json(links_df, node_df)

{'nodes': [{'id': 'Yakut', 'size': 0, 'type': 'aural'},
  {'id': 'Fiji Hindi', 'size': 0, 'type': 'aural'},
  {'id': 'Breton', 'size': 0, 'type': 'aural'},
  {'id': 'Sranan Tongo', 'size': 0, 'type': 'aural'},
  {'id': 'Ket', 'size': 0, 'type': 'aural'},
  {'id': 'Karachay-Balkar', 'size': 0, 'type': 'aural'},
  {'id': 'Skolt Sami', 'size': 0, 'type': 'aural'},
  {'id': 'Tswana', 'size': 0, 'type': 'aural'},
  {'id': 'Prussian', 'size': 0, 'type': 'aural'},
  {'id': 'Min Nan Chinese', 'size': 0, 'type': 'aural'},
  {'id': '?Xóõ', 'size': 0, 'type': 'aural'},
  {'id': 'Dongxiang', 'size': 0, 'type': 'aural'},
  {'id': 'Lezghian', 'size': 0, 'type': 'aural'},
  {'id': 'Croatian', 'size': 0.1, 'type': 'digital'},
  {'id': 'Dolgan', 'size': 0, 'type': 'aural'},
  {'id': 'Tuvinian', 'size': 0, 'type': 'aural'},
  {'id': 'Gagauz', 'size': 0, 'type': 'aural'},
  {'id': 'Khakas', 'size': 0, 'type': 'aural'},
  {'id': 'Nepali', 'size': 0, 'type': 'aural'},
  {'id': 'Wu Chinese', 'size': 0, 'typ